# Does The Sampling Method Matter?
#### Author: JP Melo

In this file we explore how different sampling method change the convergence speed and results.

### Imports

In [ ]:
from derpinns.nn import *
from derpinns.utils import *
from derpinns.trainer import *
import torch

## Parameters

In [ ]:
# Fix seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Global parameters
assets = 2

nn_shape = "64x3"               
device = torch.device("cpu") 
dtype = torch.float32

# Create dataset to traing over
batch_size = 500
total_iter = 1_000
boundary_samples = 20_000
interior_samples = boundary_samples*assets*2
initial_samples = boundary_samples*assets*2

# Define option valuation params
params = OptionParameters(
    n_assets=assets,
    tau=1.0,
    sigma=np.array([0.2] * assets),
    rho=np.eye(assets) + 0.25 * (np.ones((assets, assets)) - np.eye(assets)),
    r=0.05,
    strike=100,
    payoff=payoff
)

## Training

### With PRN

In [ ]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32
).apply(weights_init).to(device)
model.train()

sampler = "pseudo"               
dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

# we use the same optimizer for both cases
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)\
    .with_epochs(total_iter)\

trainer.train()

In [ ]:
with_pseudo = trainer.closure.get_state()
plot_loss(with_pseudo, smooth=True, smooth_window=10)

with_pseudo_results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", with_pseudo_results*100)

## With Sobol

In [ ]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32
).apply(weights_init).to(device)
model.train()

sampler = "Sobol"               
dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

# we use the same optimizer for both cases
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)\
    .with_epochs(total_iter)\

trainer.train()

In [ ]:
with_sobol = trainer.closure.get_state()
plot_loss(with_sobol, smooth=True, smooth_window=10)

with_sobol_results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", with_sobol_results*100)

## With Halton

In [ ]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32
).apply(weights_init).to(device)
model.train()

sampler = "Halton"               
dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

# we use the same optimizer for both cases
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)\
    .with_epochs(total_iter)\

trainer.train()

In [ ]:
with_halton = trainer.closure.get_state()
plot_loss(with_halton, smooth=True, smooth_window=10)

with_halton_results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", with_halton_results*100)

## Residual Based Adaptive Sampling

In [ ]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32
).apply(weights_init).to(device)
model.train()

sampler = "Sobol"               
dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

# we use the same optimizer for both cases
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

closure = ResidualBasedAdaptiveSamplingDimlessBS(sampler=sampler, k=0.5, c=2, seed=0)\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)\
    .with_epochs(total_iter)\

trainer.train()

In [ ]:
with_ra = trainer.closure.get_state()
plot_loss(with_ra, smooth=True, smooth_window=10)

with_ra_results = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", with_ra_results*100)

9.5457487/3.9504097/11.257896

### Compare runs

In [ ]:
compare_loss_histories(
    [with_pseudo,with_sobol,with_halton, with_ra],
    ["Pseudo", "Sobol", "Halton", "RAS"],
    smooth=True,
    smooth_window=50,
)

The choice of sampling method has an impact in the final achieved loss. Contrary to the intuition, the pseudo random numbers outperform all methods.

In [ ]:
print("Boundary: ", (1 - with_sobol['boundary_loss'][-1]/with_pseudo['boundary_loss'][-1])*100)
print("Interior: ", (1 - with_sobol['interior_loss'][-1]/with_pseudo['interior_loss'][-1])*100)
print("Initial: ", (1 - with_sobol['initial_loss'][-1]/with_pseudo['initial_loss'][-1])*100)


print("Boundary: ", (1 - with_ra['boundary_loss'][-1]/with_pseudo['boundary_loss'][-1])*100)
print("Interior: ", (1 - with_ra['interior_loss'][-1]/with_pseudo['interior_loss'][-1])*100)
print("Initial: ", (1 - with_ra['initial_loss'][-1]/with_pseudo['initial_loss'][-1])*100)